In [ ]:
import datetime as dt

In [ ]:
t, P = np.loadtxt('/Users/hamilton/Downloads/PRESSURE1.txt', comments='\x00').T
plot(t,P)
yscale('log')
time = np.array([dt.datetime.fromtimestamp(t[i]) for i in range(len(t))])


In [ ]:
ok = t > 1.7127e9
t = t[ok]
P = P[ok]
time = time[ok]

In [ ]:
plot(time,P)
yscale('log')

In [ ]:
dP_dt = np.gradient(P, t)
plot(t, dP_dt)
xlabel('Date')
ylabel('dP/dt [mBars/sec]')
ylim(-1e-4, 1e-4)

In [ ]:
# This code is from ChatGPT ;-)
from scipy.signal import find_peaks, find_peaks_cwt
def detect_pressure_rises(t, P, threshold, min_region_length):
    # Calculate the time derivative of pressure
    dP_dt = np.gradient(P, t)

    # Find peaks in the derivative array
    peaks, _ = find_peaks(dP_dt, height=threshold, distance=min_region_length)

    # Initialize regions list
    regions = []

    # Find rising segments starting from the peaks
    for peak_idx in peaks:
        # Find the end of the rising segment
        end_idx = peak_idx + 1
        while end_idx < len(dP_dt) and dP_dt[end_idx] > 0:
            end_idx += 1
        
        # Add the rising segment to the regions list if it exceeds min_region_length
        if t[end_idx - 1] - t[peak_idx] >= min_region_length:
            regions.append((peak_idx, end_idx - 1))

    # Convert indices to time intervals
    time_regions = [(t[start], t[end]) for start, end in regions]

    return time_regions



In [ ]:
from scipy.ndimage import gaussian_filter1d

threshold = 3e-5
min_length = 10

kernel = 100

regions = detect_pressure_rises(t, gaussian_filter1d(P, kernel), threshold, min_length)
print("Regions where pressure rises more than the threshold:")
# for region in regions:
#     print(f"Start: {region[0]}, End: {region[1]}, Duration: {region[1]-region[0]}")
    


subplot(2,1,1)
plot(t-t[0], dP_dt)
axhline(y=threshold,ls='--', color='k')
xlabel('Date')
ylabel('dP/dt [mBars/sec]')
ylim(-1e-4, 1e-4)
for r in regions:
    axvline(x=r[0]-t[0], color='r', alpha=0.3)
    axvline(x=r[1]-t[0], color='g', alpha=0.3)

xlim(50000, 60000)

subplot(2,1,2)
plot(t-t[0], P)
xlabel('Date')
ylabel('P [mBars]')
for r in regions:
    axvline(x=r[0]-t[0], color='r', alpha=0.3)
    axvline(x=r[1]-t[0], color='g', alpha=0.3)
yscale('log')
tight_layout()


In [ ]:
tinit = np.zeros(len(regions))
meanderivative = np.zeros(len(regions))
for r,i in zip(regions, arange(len(regions))):
    this = (t >= r[0]) & (t <= r[1])
    meanderivative[i] = np.mean(np.gradient(P[this], t[this]))
    tinit[i] = t[this][0]
    plot(t[this]-r[0], P[this])
yscale('log')

In [ ]:
fig, ax = plt.subplots(1)
fig.autofmt_xdate()

dateinit = np.array([dt.datetime.fromtimestamp(tinit[i]) for i in range(len(tinit))])
plot(dateinit, meanderivative, 'ro-')
yscale('log')
ylabel('Pressure increase mBars/sec')

tight_layout()